In [ ]:
#Now to supervised learning part firstly we need to perform the same data preprocessing and cleaning as earlier
def cleaning_data(train_data):
    columns_to_drop = ['ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4', 'EXTSEL992', 'KK_KUNDENTYP']
    train_data.drop(columns = columns_to_drop, inplace=True)
    k = 0
    for i,j in var_unknown.items():
        if i in columns_to_remove:
            continue
        if i in columns_to_rename:
            train_data[columns_renamed[k]] = train_data[columns_renamed[k]].replace(j, np.nan)
            continue
        train_data[i] = train_data[i].replace(j, np.nan)
    #'RELAT_AB' need to replace 9 with nan
    train_data['RELAT_AB'] = train_data['RELAT_AB'].replace([9], np.nan)
    columns_to_drop_1 = ['AGER_TYP', 'KBA05_BAUMAX', 'TITEL_KZ']
    train_data.drop(columns = columns_to_drop_1, inplace=True)

    #dropping 'LP_STATUS_GROB', 'LP_FAMILIE_GROB' because similar to 'LP_STATUS_FEIN' and 'LP_FAMILIE_FEIN' 
    train_data.drop(columns = ['LP_STATUS_GROB', 'LP_FAMILIE_GROB'], inplace=True)
    #dropping LNR
    train_data.drop(columns = ['LNR'], inplace=True)

    #reengineering features
    #PLZ8_BAUMAX, how many family homes
    train_data['PLZ8_BAUMAX'] = train_data['PLZ8_BAUMAX'].replace([5], 0)
    #if person lives in business building or not
    train_data['PLZ8_BAUMAX_BIZ'] = train_data['PLZ8_BAUMAX'].replace([0,1,2,3,4], [1,0,0,0,0])
    #main of avant-garde
    train_data['PRAEGENDE_MAIN_OR_AVANT'] = train_data['PRAEGENDE_JUGENDJAHRE'].replace([i for i in range(1,16)], [0,1,0,1,0,1,1,0,1,0,1,0,1,0,1])
    #40ies,50ies,60ies,70ies,80ies,90ies movements
    train_data['PRAEGENDE_JUGENDJAHRE'] = train_data['PRAEGENDE_JUGENDJAHRE'].replace([i for i in range(1,16)], [1,1,2,2,3,3,3,4,4,5,5,5,5,6,6])

    #CAMEO_INTL
    train_data['CAMEO_INTL_2015'] = train_data['CAMEO_INTL_2015'].replace(['X', 'XX'], np.nan)
    train_data['CAMEO_INTL_2015'] = train_data['CAMEO_INTL_2015'].astype(float)
    train_data['CAMEO_DEUG_2015'] = train_data['CAMEO_DEUG_2015'].replace(['X', 'XX'], np.nan)
    train_data['CAMEO_DEUG_2015']  = train_data['CAMEO_DEUG_2015'].astype(float)
    #household income
    train_data['CAMEO_HOUSEHOLD'] = train_data['CAMEO_INTL_2015'].replace([11, 12, 13, 14, 15, 21, 22, 23, 24, 25, 31, 32, 33, 34, 35,
                                                                                  41, 42, 43, 44, 45, 51, 52, 53, 54, 55],[1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5])
    #family type 
    train_data['CAMEO_FAMILY_TYPE'] = train_data['CAMEO_INTL_2015'].replace([11, 12, 13, 14, 15, 21, 22, 23, 24, 25, 31, 32, 33, 34, 35,
                                                                                   41, 42, 43, 44, 45, 51, 52, 53, 54, 55], [1,2,3,4,5,1,2,3,4,5,1,2,3,4,5,1,2,3,4,5,1,2,3,4,5])

    #OST_WEST_KZ encode 'W' to 0 and '0' to 1
    train_data['OST_WEST_KZ'] = train_data['OST_WEST_KZ'].replace(['W', 'O'], [0,1]) 
    #one hot encoding of 'ANREDE_KZ'
    train_data = pd.get_dummies(train_data, columns=['ANREDE_KZ'], drop_first=True)


    train_data.drop(columns = ['CAMEO_INTL_2015'], inplace=True)
    #for WOHNLAGE, replace 0 with nan
    train_data['WOHNLAGE'] = train_data['WOHNLAGE'].replace([0], np.nan)
    #create new WOHNLAGE feature, urban or rural
    train_data['WOHNLAGE_REGION'] = train_data['WOHNLAGE'].replace([1,2,3,4,5,7,8],[0,0,0,0,0,1,1])

    #dropping CAMEO_DEU, too many categories 
    train_data.drop(columns = ['CAMEO_DEU_2015'], inplace=True)
    #dropping LP_LEBENSPHASE_FEIN, too many categories
    train_data.drop(columns = ['LP_LEBENSPHASE_FEIN'], inplace=True)
    #dropping D19_LETZTER_KAUF_BRANCHE, duplicated information
    train_data.drop(columns=['D19_LETZTER_KAUF_BRANCHE'], inplace=True)

    #converting 'EINGEFUEGT_AM' to total months
    eingefuegt = np.empty(shape=(train_data.shape[0],1))
    k = 0
    train_data['EINGEFUEGT_AM'] = train_data['EINGEFUEGT_AM'].fillna(-1)
    for i in train_data['EINGEFUEGT_AM']:
        if i != -1:

            yrmthday, zeros = i.split()
            mth_1 = (2019 - (int(yrmthday[0:4]) + 1)) * 12
            mth_2 = 12 - int(yrmthday[5:6])
            mth_3 = (30 - int(yrmthday[8:10])) / 30
            mth_total = mth_1 + mth_2 + mth_3
            eingefuegt[k] = mth_total
        else:
            eingefuegt[k] = -1
        k += 1




    train_data.drop(columns =['EINGEFUEGT_AM'] , inplace=True)
    train_data['EINGEFUEGT_MODIFIED'] = pd.DataFrame(eingefuegt)

    train_data['EINGEFUEGT_MODIFIED'] = train_data['EINGEFUEGT_MODIFIED'].replace([-1],train_data['EINGEFUEGT_MODIFIED'].mode().values)
    
    return train_data
    
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';', index_col=0)
mailout_train = cleaning_data(mailout_train)
mailout_train['TARGET'] = mailout_train['RESPONSE']
mailout_train.drop(columns=['RESPONSE'], inplace=True)

#I would need to read in MAILOUT TEST and perform the same cleaning and preprocessing steps as before
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';', index_col=0)
mailout_test = cleaning_data(mailout_test)

In [ ]:
#inspecting MAILOUT TRAIN, there is only 532 positive responses, hence this is an imbalanced dataset
print(mailout_train.shape[0])
print(np.count_nonzero(mailout_train['TARGET'] == 1))

#firstly we will train a model using MAILOUT TRAIN as the training data and 'TARGET' variable as the target variable
#I will only mention XGBoost here since it gives me the highest AUC score in Kaggle competition

# I will be using bayesian optimisation to find the hyperparameter combination that give the best cross-validation mean score which is then uploaded to Kaggle.
import ast
import csv
import inspect
import sys
import time
from time import gmtime, strftime
from timeit import default_timer as timer
import numpy as np
import pandas as pd
import xgboost as xgb
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import pickle
import tensorflow_addons as tfa
predictions_all = pd.DataFrame()

from sklearn.model_selection import (
    KFold,
    train_test_split,
    GridSearchCV,
    StratifiedKFold,
    cross_val_score)

#used to name the csv file for recording cross validation results and hyperparameters combination
results_file = "bayes_optimization_summary_{}.csv".format(
    strftime("%y%m%d%H%M", gmtime())
)

# add file header, we will record the cross validation results along with the hyperparameters in a csv file
with open(results_file, "w", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(
        [
            "iteration",
            "loss",
            "overall_score",
            "mean_score",
            "std_score",
            "min_score",
            "max_score",
            "train_time",
            "params",
        ]
    )

In [ ]:
def objective(space):

    global ITERATION
    ITERATION += 1
    start = timer()
    global prediction_all

    xgbcl = xgb.XGBClassifier(colsample_bytree =  space['colsample_bytree'], gamma = space['gamma'], learning_rate = space['learning_rate'], max_depth = space['max_depth'], min_child_weight = space['min_child_weight'], n_estimators = space['n_estimators'], reg_alpha = space['reg_alpha'], scale_pos_weight = space['scale_pos_weight'], subsample = space['subsample'], tree_method = 'gpu_hist', sampling_method = 'gradient_based')

    # execute cross-validation scoring, I will used stratified cross-validation and 3 cross-validation folds
    # scoring will be roc_auc since this is an imbalanced dataset
    cv = cross_val_score(
        estimator=xgbcl,
        X=mailout_train.iloc[:,:-1],
        y=mailout_train.iloc[:,-1],
        verbose=True,
        n_jobs=-1,
        cv=StratifiedKFold(n_splits=3, shuffle=False, random_state=None),
        scoring='roc_auc',
    )
    #this fit the XGBClassifier to the MAILOUT TRAIN and fitted model is used to predict on MAILOUT TEST and predictions are concatenated to predictions_all variable
    xgbcl.fit(mailout_train.iloc[:,:-1], mailout_train.iloc[:,-1])
    predictions_xgbcl = (xgbcl.predict_proba(mailout_test))[:, 1]
    predictions_all = pd.concat([predictions_all, predictions_xgbcl], axis=1)
    

    # log runtime
    run_time = timer() - start
    # calculate loss, mean cross validation score, standard deviation and minimum and maximum cross-validation score
   
    loss = 1 - cv.mean()
    mean_score = cv.mean()
    std_score = cv.std()
    min_score = cv.min()
    max_score = cv.max()

    # export results to csv
    out_file = results_file
    with open(out_file, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(
            [
                ITERATION,
                
                loss,
                mean_score,
                std_score,
                min_score,
                max_score,
                run_time,
                space,
            ]
        )

    return loss

#this is the hyperparameter search space for XGBoost Classifier
space = { 'learning_rate': hp.uniform('learning_rate', 0.001, 0.050),
'max_depth': hp.choice('max_depth', np.arange(3, 10, 1, dtype=int)),
'min_child_weight': hp.choice('min_child_weight', np.arange(1, 20, 1, dtype = int)),
'gamma': hp.uniform('gamma', 0.04, 0.4),
'subsample': hp.uniform('subsample', 0.5, 0.8),
'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
'reg_alpha': hp.uniform('reg_alpha', 0.002 , 0.04),
'n_estimators': hp.choice('n_estimators', np.arange(400, 600,1, dtype = int)),
'scale_pos_weight': hp.choice('scale_pos_weight', np.arange(10, 50, 1, dtype=int)),
'tree_method' :'gpu_hist',
'sampling_method' : 'gradient_based'
          }

In [ ]:
#this line of codes initiates the bayesian optimisation procedure and I will set maximum iterations at 300
ITERATION=0
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=300,
    trials=Trials(),
    show_progressbar=True,
)
#this saves all the predictions on MAILOUT TEST to a csv file
pd.DataFrame(predictions_all).to_csv('xgbcl model predictions probability on MAILOUT TEST.csv')

In [ ]:
#following is the hyperparameter combination that provides the best result in Kaggle
hyperparam = {'colsample_bytree': 0.5027067891629291, 'gamma': 0.0597643487533388, 'learning_rate': 0.001370272785081644, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 505, 'reg_alpha': 0.005424468447439956, 'sampling_method': 'gradient_based', 'scale_pos_weight': 40, 'subsample': 0.46899673182868984, 'tree_method': 'gpu_hist'}

xgbcl4 = xgb.XGBClassifier(colsample_bytree =  0.5027067891629291, gamma = 0.0597643487533388, learning_rate = 0.001370272785081644, max_depth = 4, min_child_weight = 1, n_estimators = 505, reg_alpha = 0.005424468447439956, scale_pos_weight = 40, subsample = 0.46899673182868984, tree_method = 'gpu_hist', sampling_method = 'gradient_based')
xgbcl4.fit(mailout_train.iloc[:,:-1], mailout_train.iloc[:,-1])

import pickle
pickle.dump(xgbcl4, open("best xgbcl model for training on MAILOUT TRAIN.pkl" ,"wb"))

In [ ]:
#I have tried various methods of raising my AUC score in the Kaggle competition and I found that choosing the 130 most important features as predicted by the above best xgbcl
#model as features and using AZDIAS + CUSTOMER as the training dataset and MAILOUT TRAIN as the validation dataset, I can achieve the highest AUC score in Kaggle competition

#Using the K-Means analysis above for 60 most important features, i will label cluster 0 in AZDIAS as 1 (positive response) and the rest as 0. 
#while examples in CUSTOMER will all be labelled 0. I choose not to drop any row because I find that not dropping any row raises my AUC score on validation dataset

#Therefore we read the raw AZDIAS and CUSTOMER dataset again and perform the same preprocessing and cleaning steps as before
train_data = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';', index_col=0)
customer_data = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';', index_col=0)

#invoking the function that cleans and preprocesses AZDIAS and CUSTOMER dataset
# we will not fill missing values with mode/mean since XGBoost can handle missing values
train_data = cleaning_data(train_data)
customer_data = cleaning_data(customer_data)

#we use our best XGBoost model earlier to determine the 130 most important features which will be used as features in the combined AZDIAS and CUSTOMER dataset.

#this sorts features from most important to least important
sorted = np.argsort(xgbcl4.feature_importances_)[::-1]

#this appends the feature names starting from most important to least important to impt_variables list 
impt_variables = []
column_names = train_data.columns 
for i in sorted:
    impt_variables.append(column_names[i])
impt_130 = impt_variables[:130]

#this converts the ASDIAZ and CUSTOMER data earlier to only include the 130 most important features
train_data = train_data[impt_130]
customer_data = customer_data[impt_130]

#since our earlier K-Means analysis identified cluster 0 as potential customers in AZDIAS, we will label cluster 0 as 1 (positive response)
#and the other clusters as 0 (negative response)
predicted_label_azdias = pd.DataFrame(predicted_cluster_azdias1).replace([i for i in range(1,10)] ,[0 for i in range(0,9)])
predicted_label_azdias = (predicted_label_azdias).replace([0],1)
#for CUSTOMER dataset, we will label all examples as 0
predicted_label_customer = np.array(customer_data.shape[0])

# this concatenate both the AZDIAS and CUSTOMER label
predicted_label = pd.concat([pd.DataFrame(predicted_label_azdias), pd.DataFrame(predicted_label_customer)], axis=0)
#this concatenate both AZDIAS and CUSTOMER dataset along with the label
train_data = pd.concat([train_data, customer_data], axis=0)
train_data = pd.concat([train_data.reset_index(drop=True), predicted_label], axis=1)
#now we convert the MAILOUT TRAIN to only include the 130 most important features
mailout_train = mailout_train[impt_130]

In [ ]:
# We will use Bayesian Optimisation again. For the hyperparameter max_depth, after conducting Bayesian Optimisation for many hundreds iterations, I found that max_depth 16 yields the best AUC score in validation dataset
#as such max_depth range will only be set to 16 and as for other hyperparameters, I progressively narrowed my search range by looking up trained models with hyperparameter combinations that yield the best AUC score in both mailout train and mailout test. 
#As such, I was able to arrive at a hyperparameters combination with max_depth of 16 that produce the highest AUC score in the leaderboard, 0.88149. 
#I have tried about 30 more times to best my result. However, my effort was futile and I persuaded myself to take satisfaction in my current result. 


import ast
import csv
import inspect
import sys
import time
from time import gmtime, strftime
from timeit import default_timer as timer
import numpy as np
import pandas as pd
import xgboost as xgb
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import pickle
import tensorflow_addons as tfa

predictions_all_1 = pd.DataFrame()
validation_data = mailout_train.copy()

#I will split validation data(MAILOUT TRAIN) into three parts because in the Kaggle competition, it states only 30% of MAILOUT TEST will be used for evaluation
# I also have to ensure each part has equal number of RESPONSE of 1 since this is an imbalanced dataset.
mailout_train_zeros = validation_data[validation_data['TARGET']==0]
mailout_train_ones = validation_data[validation_data['TARGET']==1]
mailout_train_sample1 = np.array(pd.concat([mailout_train_zeros.iloc[:13000,:],mailout_train_ones.iloc[:177,:]]).sample(frac=1, random_state =42))
mailout_train_sample2 = np.array(pd.concat([mailout_train_zeros.iloc[13000:26000,:],mailout_train_ones.iloc[177:354,:]]).sample(frac=1, random_state =42))
mailout_train_sample3 = np.array(pd.concat([mailout_train_zeros.iloc[26000:42962,:],mailout_train_ones.iloc[354:532,:]]).sample(frac=1, random_state =42))
validation_data = np.array(validation_data)

#used to name the csv file for recording cross validation results and hyperparameters combination
results_file = "bayes_optimization_summary_{}.csv".format(
    strftime("%y%m%d%H%M", gmtime())
)

# add file header, we will record the cross validation results along with the hyperparameters in a csv file
with open(results_file, "w", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(
        [
            "iteration",
            "loss",
            "overall_score",
            "mean_score",
            "std_score",
            "min_score",
            "max_score",
            "train_time",
            "params",
        ]
    )
    
def objective(space):

    global ITERATION
    ITERATION += 1
    start = timer()
    global prediction_all


    # used to store the auc scores for all three parts of MAILOUT TRAIN.
    auc_score = np.empty(3)
    
    #training the XGBClassifier on the combined AZDIAS and CUSTOMER dataset with MAILOUT TRAIN as the validation dataset and early stopping rounds set to 40 
    eval_set = [(validation_data[:, :-1], validation_data[:, -1])]
    xgbcl = xgb.XGBClassifier(colsample_bytree =  space['colsample_bytree'], gamma = space['gamma'], learning_rate = space['learning_rate'], max_depth = space['max_depth'], min_child_weight = space['min_child_weight'], n_estimators = space['n_estimators'], reg_alpha = space['reg_alpha'], scale_pos_weight = space['scale_pos_weight'], subsample = space['subsample'], tree_method = 'gpu_hist', sampling_method = 'gradient_based')
    xgbcl.fit(np.array(train_data.iloc[:,:-1]), np.array(train_data.iloc[:,-1]), early_stopping_rounds=40, eval_metric="auc", eval_set=eval_set)
    
    #trained XGBClassifier model is then used to predict on overall MAILOUT TRAIN data and also the 3 separate parts
    prediction = (xgbcl.predict_proba(validation_data[:,:-1]))[:,1]
    score_a = roc_auc_score(validation_data[:,-1], prediction)
    predictions1 = (xgbcl.predict_proba(mailout_train_sample1[:, :-1]))[:, 1]
    auc_score[0]=roc_auc_score(mailout_train_sample1[:, -1], predictions1)
    predictions2 = (xgbcl.predict_proba(mailout_train_sample2[:, :-1]))[:, 1]
    auc_score[1]=roc_auc_score(mailout_train_sample2[:, -1], predictions2)
    predictions3 = (xgbcl.predict_proba(mailout_train_sample3[:, :-1]))[:, 1]
    auc_score[2]=roc_auc_score(mailout_train_sample3[:, -1], predictions3)
    
     
    #then trained XGBClassifier model is used to predict on MAILOUT TEST and predictions are concatenated to predictions_all_1 variable  
    predictions_xgbcl = (xgbcl.predict_proba(mailout_test))[:, 1]
    predictions_all_1 = pd.concat([predictions_all, predictions_xgbcl], axis=1)
    
   
    # log runtime
    run_time = timer() - start

    # calculate mean loss, mean, min and max auc scores over the 3 parts and overall auc scores on the whole MAILOUT TRAIN dataset

    loss = 1 - auc_score.mean()
    mean_score = auc_score.mean()
    std_score = auc_score.std()
    min_score = auc_score.min()
    max_score = auc_score.max()
    overall_score = score_a

    # export results to csv
    out_file = results_file
    with open(out_file, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(
            [
                ITERATION,
                loss,
                overall_score,
                mean_score,
                std_score,
                min_score,
                max_score,
                run_time,
                space
            ]
        )

    return loss

In [ ]:
#I progressively narrowed my search range by looking up trained models with hyperparameter combinations that yield the best AUC score in both mailout train and mailout test. 
#Below is the refined search space for hyperparameters combination that yield the best AUC score on validation dataset.
#As such, I was able to arrive at a hyperparameters combination with max_depth of 16 that produce the highest AUC score in the leaderboard, 0.88149.

space = { 'learning_rate': hp.uniform('learning_rate', 0.05, 0.13),
'max_depth': hp.choice('max_depth', np.arange(16, 17, 1, dtype=int)),
'min_child_weight': hp.choice('min_child_weight', np.arange(5, 9, 1, dtype = int)),
'gamma': hp.uniform('gamma', 0.9, 1.1),
'subsample': hp.uniform('subsample', 0.8, 1.0),
'colsample_bytree': hp.uniform('colsample_bytree', 0.7, 0.85),
'reg_alpha': hp.uniform('reg_alpha', 0.17 , 0.23),
'n_estimators': hp.choice('n_estimators', np.arange(500, 700,1, dtype = int)),
'scale_pos_weight': hp.choice('scale_pos_weight', np.arange(45, 55, 1, dtype=int)),
'tree_method' :'gpu_hist',
'sampling_method' : 'gradient_based'
          }

In [ ]:
#this line of codes initiates the bayesian optimisation procedure and I will set maximum iterations at 200
ITERATION=0
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=200,
    trials=Trials(),
    show_progressbar=True,
)

In [ ]:
#following is the hyperparameters combination that provides the best result in Kaggle, namely an AUC of 0.88149
hyperparam_best = {'colsample_bytree': 0.776792351030805, 'gamma': 1.0386435777290857, 'learning_rate': 0.10327932030577727, 'max_depth': 16, 'min_child_weight': 5, 'n_estimators': 571, 'reg_alpha': 0.19625707362167116, 'sampling_method': 'gradient_based', 'scale_pos_weight': 48, 'subsample': 0.8894945842180019, 'tree_method': 'gpu_hist'}

#training XGBClassifier using the best hyperparameters combination on the combined AZDIAS and CUSTOMER dataset
xgbcl5 = xgb.XGBClassifier(colsample_bytree = 0.776792351030805, gamma = 1.0386435777290857, learning_rate = 0.10327932030577727, max_depth = 17, min_child_weight = 5, n_estimators = 571, reg_alpha = 0.19625707362167116, scale_pos_weight = 48, subsample = 0.8894945842180019, tree_method = 'gpu_hist', sampling_method = 'gradient_based')
# as before i used MAILOUT TRAIN as my validation dataset with early stopping rounds set at 40.
eval_set = [(mailout_train[:, :-1], mailout_train[:, -1])]
xgbcl5.fit(np.array(train_data.iloc[:,:-1]), np.array(train_data.iloc[:,-1]), early_stopping_rounds=40, eval_metric="auc", eval_set=eval_set)
#find the AUC for predictions on MAILOUT TRAIN
prediction = (xgbcl5.predict_proba(mailout_train[:,:-1]))[:,1]
score_auc = roc_auc_score(mailout_train[:,-1], prediction)

#use the trained XGBClassifier model to predict on MAILOUT TEST and which will be submitted to Kaggle competition
#convert MAILOUT TEST to only include the 130 most important features
mailout_test = mailout_test[impt_130]

#would need to obtain the LNR column for submission to Kaggle
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', index_col=0)
mailout_test_lnr = mailout_test['LNR']

#predict on the MAILOUT TEST
predictions_xgbcl = (xgbcl.predict_proba(mailout_test))[:, 1]
#concatenate predictions with LNR for submission to Kaggle
predictions_xgbcl = pd.concat([mailout_test_lnr, predictions_xgbcl], axis = 1, ignore_index = True)